<H3> Import Libraries

In [1]:
from read_user_timeline import read_user_timeline
import pickle
import os
import pandas as pd

<H3> Define the microsoft handles to get tweets from

In [20]:
office = ['Microsoft365','MSFTDynamics365','linkedin']
computing = ['Xbox','surface','windows']
cloud = ['Azure','SQLServer','windowsserver','github']

<H3> Get the tweets

In [4]:
tweets_office=[]
for thandle in office:
    if(os.path.isfile(thandle+'tweets.sav')==False):
        print('Extracting timeline for ' +thandle)
        read_user_timeline(thandle, 'tweepy')
        tweets_office.append(pickle.load(open(thandle+'tweets.sav','rb')))
    else:
        print('Loading previously extracted timeline for ' + thandle)
        tweets_office.append(pickle.load(open(thandle+'tweets.sav','rb')))

Loading previously extracted timeline for Microsoft365
Loading previously extracted timeline for MSFTDynamics365
Loading previously extracted timeline for linkedin


<H3> Process the teweets
<H4>extract favourite count, retweet count
<H4>remove tweets that are replies

In [23]:
date_range = []
date_list = []
text_list = []
reply_list = []
fvt_list=[]
rt_list=[]
for i in range(len(tweets_office)):
    dates = [t['created_at'] for t in tweets_office[i]]
    dates = pd.to_datetime(dates)
    date_list.append(dates)
    text = [t['text'] for t in tweets_office[i]]
    text_list.append(text)
    fvt_list.append([a['favorite_count'] for a in tweets_office[i]])
    rt_list.append([a['retweet_count'] for a in tweets_office[i]])
    reply_list.append([t['in_reply_to_status_id'] for t in tweets_office[i]])

    tmp=[]
    tmp.append(dates.min())
    tmp.append(dates.max())
    date_range.append(tmp)

In [24]:
date_range

[[Timestamp('2020-02-20 12:28:22+0000', tz='UTC'),
  Timestamp('2020-09-17 18:00:05+0000', tz='UTC')],
 [Timestamp('2019-08-21 20:46:00+0000', tz='UTC'),
  Timestamp('2020-09-17 18:30:02+0000', tz='UTC')],
 [Timestamp('2018-02-23 23:48:00+0000', tz='UTC'),
  Timestamp('2020-09-17 16:18:35+0000', tz='UTC')]]

In [25]:
date_list[0].shape

(2000,)

In [26]:
df = pd.DataFrame(date_list[0], columns=['Date'])   
df['Text'] = text_list[0]
df['fv_cnt'] = fvt_list[0]
df['rt_cnt'] = rt_list[0]
df['reply'] = reply_list[0]
df = df[df.reply.isnull()]
df.drop('reply', axis=1, inplace=True)
df = df.sort_values('Date').reset_index(drop=True)
df.head(10)

,Date,Text,fv_cnt,rt_cnt
0,2020-02-20 14:00:00+00:00,Do tensions arise in your teams? Are you able ...,37,17
1,2020-02-20 16:00:01+00:00,"#Word, @MSExcel, @PowerPoint in one app. The n...",427,129
2,2020-02-20 18:15:00+00:00,Chances are your organization feels susceptibl...,231,86
3,2020-02-20 20:12:00+00:00,Leverage #AI and #machinelearning to address i...,131,45
4,2020-02-20 22:00:00+00:00,Discover how @Microsoft is speeding up @Window...,10,1
5,2020-02-20 23:00:00+00:00,Take action on code-of-conduct policy violatio...,93,41
6,2020-02-21 00:00:00+00:00,"Looking for greater bandwidth to activity API,...",24,7
7,2020-02-21 16:00:02+00:00,Convert an image of a table into an @MSExcel s...,80,30
8,2020-02-21 17:59:31+00:00,RT @CheddarNews: #Microsoft unveils its new 'O...,0,10
9,2020-02-21 20:35:00+00:00,Empower #SMB customers with security and produ...,12,3


In [27]:
df_monthly = df.copy()
df_monthly.index = df.Date
df_monthly = df_monthly.resample('MS')[['fv_cnt','rt_cnt']].mean().reset_index()
df_monthly

,Date,fv_cnt,rt_cnt
0,2020-02-01 00:00:00+00:00,74.142857,24.942857
1,2020-03-01 00:00:00+00:00,209.339623,98.075472
2,2020-04-01 00:00:00+00:00,84.696970,65.969697
3,2020-05-01 00:00:00+00:00,68.476190,57.210884
4,2020-06-01 00:00:00+00:00,63.086022,148.322581
5,2020-07-01 00:00:00+00:00,59.706294,36.461538
6,2020-08-01 00:00:00+00:00,58.710526,35.043860
7,2020-09-01 00:00:00+00:00,36.015873,30.285714


In [29]:
import plotly.express as px

In [30]:
fig = px.line(df_monthly, x="Date", y="fv_cnt", title='Average Favourite Count')
fig.show()

In [31]:
fig = px.line(df_monthly, x="Date", y="rt_cnt", title='Average Retweet Count')
fig.show()